# Visualizing 3D data in napari
In this tutorial, you will learn:
- How to load images from hdf5 files
- How to open images in napari
- The difference between intensity images and label images
- How to visualize measurements on images using a napari plugin
- How to explore image data interactively

In [ ]:
import h5py
# We import some custom functions to handle the hdf5 files
from h5_files import h5_select, h5_summary, to_numpy

In [ ]:
# Get an overview of what's in an hdf5 file
file_path = 'PATH/TO/H5/FILE'
print(h5_summary(file_path))

In [ ]:
# Load downsampled versions of the intensity images (level 2, if you have more memory available you can load the higher resolution images (1 or 0)).
# Load the segmentation masks

img_level = 2 # 2, 1 or 0

with h5py.File(file_path) as f:
    print('extracting image scale from metadata...')
    sytox_dset = h5_select(f, {'stain': 'SYTOX', 'level': img_level})[0]
    img_scl = sytox_dset.attrs['element_size_um']
    
    print(f'loading sytox on resolution level: {img_level}...')
    sytox = to_numpy(sytox_dset)
    
    print(f'loading pcna on resolution level: {img_level}...')
    pcna = to_numpy(h5_select(f, {'stain': 'PCNA', 'level': img_level})[0])
    
    print(f'loading b-catenin on resolution level: {img_level}...')
    bcat = to_numpy(h5_select(f, {'stain': 'bCatenin', 'level': img_level})[0])
    
    print(f'loading 5-eu on resolution level: {img_level}...')
    eu = to_numpy(h5_select(f, {'stain': '5-EU', 'level': img_level})[0])
    
    print(f'loading nuclear segmentation...')
    lbl_scl = f['lbl_nuc'].attrs['element_size_um']
    nucs = to_numpy(f['lbl_nuc'])
    
    print(f'loading cell segmentation...')
    cells = to_numpy(f['lbl_cells'])
    
    print(f'loading membrane segmentation...')
    mems = to_numpy(f['lbl_mem'])

## The napari viewer
napari is a fast, interactive, multi-dimensional image viewer for Python. It’s designed for browsing, annotating, and analyzing large multi-dimensional images.  
https://napari.org

In [ ]:
import napari

In [ ]:
# Open the image and the segmentation in the napari viewer
viewer = napari.Viewer()
viewer.add_image(sytox, scale=img_scl, blending='additive', colormap='blue', contrast_limits=(0, 400))
viewer.add_image(pcna, scale=img_scl, blending='additive', colormap='red', contrast_limits=(0, 100))
viewer.add_image(bcat, scale=img_scl, blending='additive', colormap='green', contrast_limits=(0, 100))
viewer.add_image(eu, scale=img_scl, blending='additive', colormap='red', contrast_limits=(0, 200), visible=False)

viewer.add_labels(nucs, scale=lbl_scl, visible=False)
viewer.add_labels(cells, scale=lbl_scl, visible=False)
viewer.add_labels(mems, scale=lbl_scl, visible=False)

# Questions to look at during the exploration
- How does nuclear size & cell size vary over the embryo?
- How does the 5-EU signal vary over the different nuclei and what does that mean?
- How does PCNA vary over the embryo and what does that mean?
- What would you need to measure to quantitatively answer those questions?
- What potential technical biases do you see in the images that we should be aware of when quantifying them?

## Visualize feature measurements
Open the napari feature visualization plugin (plugins -> napari-feature-visualization -> feature_vis

label layer: nucs  
DataFrame: **visualization_features.csv**  
label column: Label

Using the napari feature visualization plugin, look at some of the features, e.g.:  
- Physical Size of the nuclei or the cell. Roundness
- EU intensity (mean, skewness)
- PCNA intensities (mean)  
Do the quantifications help you answer the questions above?